In [11]:
import numpy as np
from datasets import get_dataset
import sys 
sys.path.insert(0, "../")
from model_builder import get_base_model, get_pretrained_model_v2
DATASET = "lioness"
    

In [12]:
backbone, cfg = get_base_model("mae-lightning-small")
train_dataset, _, _ = get_dataset(name=DATASET, cfg=cfg)
print(len(train_dataset))

3100
